In [1]:
using JLD2
using Statistics
using Revise
using Plots
using Laplacians
using Printf

In [2]:
@load "wgrid_all.jl.split02merge02.jld2"
dic1 = copy(dic)

Dict{Any, Any} with 84 entries:
  "lamg_build"        => [363.211, 350.476, 324.422, 329.775, 337.111, 277.833,…
  "acM2C2_err"        => [6.0718e-9, 9.48525e-9, 6.55088e-9, 6.11278e-9, 5.1397…
  "hypre_solve"       => [59.37, 59.74, 64.19, 47.03, 64.66, 56.32, 44.69]
  "acM2C2_solve"      => [298.573, 259.155, 243.942, 271.495, 180.748, 107.17, …
  "icc2_tot"          => [680.159, 604.273, 555.115, 530.482, 462.795, 261.693,…
  "acM2C2_tot"        => [540.099, 504.874, 499.0, 536.09, 497.11, 299.526, 253…
  "ac_lst_build"      => [214.724, 229.054, 231.638, 218.967, 285.977, 476.579,…
  "petsc_hypre_build" => [107.746, 128.613, 431.093, 319.13, 293.971, 84.1446, …
  "lamg_its"          => [9.0, 9.0, 9.0, 8.0, 10.0, 13.0, 14.0]
  "lamg_solve"        => [458.241, 439.766, 413.963, 361.416, 478.587, 572.141,…
  "hypre_its"         => [7.0, 7.0, 8.0, 7.0, 7.0, 8.0, 7.0]
  "cmg_solve"         => [97.1844, 98.3972, 98.4658, 100.537, 102.408, 110.121,…
  "ac_tot"            => [334.537, 306.

In [3]:
function sd(d,x)
    #scale = exp10(ceil(log10(abs(x)))-d)
    #return round(x/scale)*scale
    return round(x, sigdigits=d)
end        

#constant d = 3
ff(s) = (@sprintf "%.3g" sd(3,s))
nf(s) = (@sprintf "%d" sd(3,s))

ffK(s) = (@sprintf "%.3g" sd(3,s)/1000)
nfK(s) =  round(Int64, sd(3,s)/1000)

ffe8(s) = (@sprintf "%.3g" sd(3,s)*1e8)
ffM(s) = (@sprintf "%.3g" sd(3,s)/1e6)
nfM(s) =  round(Int64, sd(3,s)/1e6)
ffe6(s) = (@sprintf "%.3g" sd(3,s)*1e6)

ffe6 (generic function with 1 method)

In [4]:
function labelTvn(tvn, err)
    if err <= 1e-8
        return ffe6(tvn)
    elseif err < 1e-4
        return string(ffe6(tvn), "\\tnote{*}")
    elseif err < 1
        return string(ffe6(tvn), "\\tnote{**}")
    else
        return "Inf"
    end
end

function printSolverCell(d, solver, i)
    if haskey(d, string(solver, "_tot"))
        itr = d[string(solver, "_its")][i]
        ne = d["ne"][i]
        solveVn = d[string(solver, "_solve")][i] / ne
        totalVn = d[string(solver, "_tot")][i] / ne
        err = d[string(solver, "_err")][i]
        return string("&", nf(itr), "&", labelTvn(solveVn, err), "&", labelTvn(totalVn, err), "&",ffe8(err))
    else
        return "&N/A&N/A&N/A&N/A"
    end
end

function nfffswitch(val)
    if val < 1
        return ff(val)
    else
        return nf(val)
    end
end

function printOneLine(d, solvers, i)
    allCells = [printSolverCell(d, solver, i) for solver in solvers]
    return string(nfffswitch(exp10(i-4)), join(allCells), "\\\\")
end

function printAllLines(d, solvers)
    for i in 1:length(d["testName"])
        println(printOneLine(d, solvers, i))
    end
end

printAllLines (generic function with 1 method)

In [5]:
solvers1 = ["ac", "ac-s2m2", "cmg2"]
printAllLines(dic1, solvers1)

0.001&39&1.09&1.67&0.902&25&1.09&2.48&0.657&28&0.568&0.979&0.645\\
0.01&33&0.951&1.53&0.944&23&0.997&2.41&0.963&27&0.55&0.974&0.83\\
0.1&26&0.778&1.37&0.992&20&0.925&2.39&0.645&28&0.573&0.983&0.619\\
1&26&0.814&1.52&0.952&20&0.991&2.5&0.846&28&0.636&1.05&0.653\\
10&20&0.571&1.21&0.818&15&0.63&1.95&0.413&31&0.857&1.27&0.785\\
100&17&0.399&0.916&0.464&12&0.378&1.51&0.612&30&0.722&1.15&0.83\\
1000&14&0.302&0.78&0.814&10&0.257&1.05&0.91&30&0.684&1.09&0.865\\


In [6]:
solvers2 = ["hypre", "petsc_hypre", "icc2"]
printAllLines(dic1, solvers2)

0.001&7&0.297&0.662&0.165&6&0.621&1.16&0.000816&210&3.25&3.4&0.991\\
0.01&7&0.299&0.683&0.159&6&0.733&1.38&0.000871&185&2.87&3.02&1\\
0.1&8&0.321&0.772&0.134&7&2.76&4.92&0.00865&169&2.63&2.78&0.97\\
1&7&0.235&0.588&0.207&5&0.861&2.46&0.00581&157&2.5&2.65&0.971\\
10&7&0.323&0.625&0.618&5&1.09&2.56&0.0704&139&2.16&2.31&0.997\\
100&8&0.282&0.449&0.34&5&0.629&1.05&0.0317&74&1.16&1.31&0.998\\
1000&7&0.223&0.332&0.128&6&0.644&0.904&0.000392&30&0.48&0.63&0.87\\
